In [31]:
import pandas as pd
import numpy as np
import json
import psycopg2
import random
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import string
import datetime
import csv


###Connect to database. Get tweets with geometry

In [32]:
conn_dict = {'dbname':'zipfiantwitter', 'user':'clwilloughby', 'password': '', 'host':'/tmp'}
conn = psycopg2.connect(dbname=conn_dict['dbname'], user=conn_dict['user'], host='/tmp')
cur = conn.cursor()

map_query = """
SELECT timestamp_ms, text, geoid10
FROM tweeted_neighb_wk2
                ;"""

df_tweet_shapes = pd.read_sql(map_query, conn)

In [35]:
df_tweet_shapes.head()

,timestamp_ms,text,geoid10
0,1438896121923,ech75,060375760011015
1,1438847519615,Tf it s already 1,060378002022025
2,1438848499256,test,060750201001001
3,1438545172798,https t co scRjn8RVmU Office prank,060759804011003
4,1438545183785,Sunday fun day lake McSwain hot BBQ beer funti...,060816013003000


In [13]:
#this gets me a count AND it gets me a 
def tokenize_tweets(text):
    '''Called by sql_tweets_sum_and_tokenized. Tokenizes a sting.'''
    punct = string.punctuation
    stop_words = stopwords.words('english')
    text_list = []
    snowball = SnowballStemmer('english')
    for word in text.lower().split():
        word = word.strip(punct)
        if word not in stop_words:
            text_list.append(snowball.stem(word))
    return text_list

def sql_tweets_sum_and_tokenized(df, timeseries= True):
    '''Takes a sql dataframe of tweets, geoids, and timestamps.
    Tokenizes the tweets. Groupsby geoid and timestamp (if True) to create overal counts
    Returns a new dataframe.'''
    #take the string and tokenize words for a list 
    df['text'] = df.text.apply(tokenize_tweets)
    #create a column of ones for quick sum column during groupby
    df.insert(1, 'count', 1)
    #groupby parameters. Counts Tweets and appends tweet tokens to a master list. 
    if timeseries == False:
        df_group = df.groupby('geoid10').apply(lambda x: x.sum())
        #drop redundant columns
        df_group.drop('geoid10', 1, inplace=True)
        df_group.drop('timestamp_ms', 1, inplace=True)
        df_group = df_group.reset_index()
    else:
        #somethign about time. This will be difficult. 
        return df
    
    return df_group

In [14]:
#df_token_counts = sql_tweets_sum_and_tokenized(df_tweet_shapes)
#df_token_counts.to_csv('tweet_count_new.csv')  

###Let's Get Time Formatted

In [40]:
df_tweet_shapes['times'] = df_tweet_shapes['timestamp_ms'].values.astype(int).astype('datetime64[ms]')
df_tweet_shapes['hour']= pd.DatetimeIndex(df_tweet_shapes["times"]).hour
df_tweet_shapes['DOW']= pd.DatetimeIndex(df_tweet_shapes["times"]).dayofweek
df_tweet_shapes['date']= pd.DatetimeIndex(df_tweet_shapes["times"]).date

In [43]:
df_tweet_shapes.head()

,timestamp_ms,count,text,geoid10,times,hour,DOW,date
0,1438896121923,1,[ech75],060375760011015,2015-08-06 21:22:01.923,21,3,2015-08-06
1,1438847519615,1,"[tf, alreadi, 1]",060378002022025,2015-08-06 07:51:59.615,7,3,2015-08-06
2,1438848499256,1,[test],060750201001001,2015-08-06 08:08:19.256,8,3,2015-08-06
3,1438545172798,1,"[https, co, scrjn8rvmu, offic, prank]",060759804011003,2015-08-02 19:52:52.798,19,6,2015-08-02
4,1438545183785,1,"[sunday, fun, day, lake, mcswain, hot, bbq, be...",060816013003000,2015-08-02 19:53:03.785,19,6,2015-08-02


In [42]:
df_timetweet = sql_tweets_sum_and_tokenized(df_tweet_shapes, True)

##limit tweets to SF County Only

In [18]:
with open('data/sf_geoids.csv', 'rb') as f:
    reader = csv.reader(f)
    sf_geolist = list(reader)[0]

In [35]:
df_timetweet = df_timetweet[df_timetweet['geoid10'].astype(str).isin(sf_geolist)]

In [42]:
df_timetweet.head()

,timestamp_ms,count,text,geoid10,times,hour,DOW,date
1,1438147171326,1,"[u'cocktail', u'love', u'arlen', u'amaz', u'su...",60750171023004,2015-07-29 05:19:31.326,5,2,2015-07-29
2,1438147180701,1,"[u'http', 'co', u'djttj3rqj2']",60750208003005,2015-07-29 05:19:40.701,5,2,2015-07-29
3,1438147182264,1,"[u'close', u'graffiti', u'request', u'intersec...",60750207002000,2015-07-29 05:19:42.264,5,2,2015-07-29
4,1438147185252,1,"[u'close', u'graffiti', u'request', u'intersec...",60750208004004,2015-07-29 05:19:45.252,5,2,2015-07-29
6,1438147189241,1,"[u'marketstreet', u'powellstreet', u'goldeng',...",60750130004001,2015-07-29 05:19:49.241,5,2,2015-07-29


In [43]:
df.to_csv('data/tweet_with_time_wk1.csv')